In [20]:
from types import SimpleNamespace
from collections import Counter
import os
import re
import pathlib
import array
import pickle
import numpy as np
import torch
import torch.nn as nn
import pandas as pd

In [21]:
import time
import numpy as np

import torch
import torch.nn as nn

from typing import Tuple, Dict, Any, List
from torchvision import datasets, transforms

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

In [22]:
DATASET_VERSION = 'ca-100'
COMPETITION_ROOT = '../input/vectors4'
DATASET_ROOT = f'../input/text-preprocessing/data/{DATASET_VERSION}'
WORKING_ROOT = f'data/{DATASET_VERSION}'
DATASET_PREFIX = 'ca.wiki'

In [23]:
params = SimpleNamespace(
    embedding_dim = 400,
    window_size = 7,
    batch_size = 1000,
    epochs = 4,
    preprocessed = f'{DATASET_ROOT}/{DATASET_PREFIX}',
    working = f'{WORKING_ROOT}/{DATASET_PREFIX}',
    modelname = f'{WORKING_ROOT}/{DATASET_VERSION}.pt',
    train = True
)

In [24]:
class Vocabulary(object):
    def __init__(self, pad_token='<pad>', unk_token='<unk>', eos_token='<eos>'):
        self.token2idx = {}
        self.idx2token = []
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_token = eos_token
        if pad_token is not None:
            self.pad_index = self.add_token(pad_token)
        if unk_token is not None:
            self.unk_index = self.add_token(unk_token)
        if eos_token is not None:
            self.eos_index = self.add_token(eos_token)

    def add_token(self, token):
        if token not in self.token2idx:
            self.idx2token.append(token)
            self.token2idx[token] = len(self.idx2token) - 1
        return self.token2idx[token]

    def get_index(self, token):
        if isinstance(token, str):
            return self.token2idx.get(token, self.unk_index)
        else:
            return [self.token2idx.get(t, self.unk_index) for t in token]

    def get_token(self, index):
        return self.idx2token[index]

    def __len__(self):
        return len(self.idx2token)

    def save(self, filename):
        with open(filename, 'wb') as f:
            pickle.dump(self.__dict__, f)

    def load(self, filename):
        with open(filename, 'rb') as f:
            self.__dict__.update(pickle.load(f))

In [ ]:
class SkipGram(nn.Module):
    def _init_(self, vocab_size, embed_size):
        super(SkipGram, self)._init_()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(embed_size, vocab_size, bias = False)
    
    def forward(self, inputs):
        embeds = self.embeddings(inputs)  
        embeds = embeds.unsqueeze(1).repeat(1, 6, 1)
        output = self.linear(embeds)  
        return output

In [25]:
def batch_generator(idata, target, batch_size, shuffle=True):
    nsamples = len(idata)
    if shuffle:
        perm = np.random.permutation(nsamples)
    else:
        perm = range(nsamples)

    for i in range(0, nsamples, batch_size):
        batch_idx = perm[i:i+batch_size]
        if target is not None:
            yield idata[batch_idx], target[batch_idx]
        else:
            yield idata[batch_idx], None

CBOW model
----------
You can add new parameters to the model in the *\_\_init\_\_()* method with *self.register_buffer()* (for parameters not to be trained):

    self.register_buffer('position_weight', torch.tensor([1,2,3,3,2,1], dtype=torch.float32))

or *nn.Parameter()* (for parameters to be trained)

    self.position_weight = nn.Parameter(torch.tensor([1,2,3,3,2,1], dtype=torch.float32))
    
In both cases, you can reference and use them in the *forward* method as

    self.position_weight

In [26]:
#ejercicio 1
class CBOW(nn.Module):
    def __init__(self, num_embeddings, embedding_dim):
        super().__init__()
        self.emb = nn.Embedding(num_embeddings, embedding_dim, padding_idx=0)
        self.lin = nn.Linear(embedding_dim, num_embeddings, bias=False)
        #Con un escalar fijo:
        #self.register_buffer('position_weight', torch.tensor([1,2,3,3,2,1], dtype=torch.float32))
        #.view(1,-1,1)
        #Con un escalar entrenado
        #self.position_weight = nn.Parameter(torch.tensor([1,2,3,3,2,1], dtype=torch.float32))
        #Con un vector entrenado:
        self.position_weight = nn.Parameter(torch.randn(6,embedding_dim), requires_grad=True)
        

    # B = Batch size
    # W = Number of context words (left + right)
    # E = embedding_dim
    # V = num_embeddings (number of words)
    def forward(self, input):
        # input shape is (B, W)
        e = self.emb(input)
        weighted_e=e*self.position_weight
        # e shape is (B, W, E)
        u = weighted_e.sum(dim=1)
        # u shape is (B, E)
        z = self.lin(u)
        # z shape is (B, V)
        return z

We can visually show how this model works:

<div style="text-align:center">
    <img src="https://drive.google.com/uc?id=1INeftgKRCS0QWZu9InCDZ42a5fph3jQX" alt="drawing" width="800" height="800"/>
</div>

We train the model to learn matrices $\mathbf{E}$ and $\mathbf{W}$. Probabilities $P(w_t|w_{c-m},\cdots,w_{c+m})$ can be obtained by applying the softmax function to the output vector $\mathbf{z}$. However, our loss function [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) takes care of this.


In [27]:
def load_preprocessed_dataset(prefix):
    # Try loading precomputed vocabulary and preprocessed data files
    token_vocab = Vocabulary()
    token_vocab.load(f'{prefix}.vocab')
    data = []
    for part in ['train', 'valid', 'test']:
        with np.load(f'{prefix}.{part}.npz') as set_data:
            idata, target = set_data['idata'], set_data['target']
            data.append((idata, target))
            print(f'Number of samples ({part}): {len(target)}')
    print("Using precomputed vocabulary and data files")
    print(f'Vocabulary size: {len(token_vocab)}')
    return token_vocab, data

In [ ]:
def load_preprocessed_dataset_SkipGram(prefix):
    # Try loading precomputed vocabulary and preprocessed data files
    token_vocab = Vocabulary()
    token_vocab.load(f'{prefix}.vocab')
    data = []
    for part in ['train', 'valid', 'test']:
        with np.load(f'{prefix}.{part}.npz') as set_data:
            target, idata = set_data['idata'], set_data['target']
            data.append((idata, target))
            print(f'Number of samples ({part}): {len(target)}')
    print("Using precomputed vocabulary and data files")
    print(f'Vocabulary size: {len(token_vocab)}')
    return token_vocab, data

In [28]:
def train(model, criterion, optimizer,scheduler, idata, target, batch_size, device, log=False):
    model.train()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    for X, y in batch_generator(idata, target, batch_size, shuffle=True):
        # Get input and target sequences from batch
        X = torch.tensor(X, dtype=torch.long, device=device)
        y = torch.tensor(y, dtype=torch.long, device=device)

        model.zero_grad()
        output = model(X)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        scheduler.step()
        # Training statistics
        total_loss += loss.item()
        ncorrect += (torch.max(output, 1)[1] == y).sum().item()
        ntokens += y.numel()
        niterations += 1
        if niterations == 200 or niterations == 500 or niterations % 1000 == 0:
            print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={100*ncorrect/ntokens:.1f}, loss={total_loss/ntokens:.2f}')

    total_loss = total_loss / ntokens
    accuracy = 100 * ncorrect / ntokens
    if log:
        print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={accuracy:.1f}, loss={total_loss:.2f}')
    return accuracy, total_loss

In [ ]:
def train_SkipGram(model, criterion, optimizer, idata, target, batch_size, device, log=False):
    model.train()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    

    for X, y in batch_generator(idata, target, batch_size, shuffle=True):
        # Get input and target sequences from batch
        X = torch.tensor(X, dtype=torch.long, device=device)
        y = torch.tensor(y, dtype=torch.long, device=device)
        
        model.zero_grad()
        output = model(X)
        
        output = output.permute(0, 2, 1)
        loss = criterion(output, y)
        loss.backward()
        optimizer.step()
        # Training statistics
        total_loss += loss.item()
        ncorrect += (torch.max(output, 1)[1] == y).sum().item()
        ntokens += y.numel()
        niterations += 1
        if niterations == 200 or niterations == 500 or niterations % 1000 == 0:
            print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={100*ncorrect/ntokens:.1f}, loss={total_loss/ntokens:.2f}')

    total_loss = total_loss / ntokens
    accuracy = 100 * ncorrect / ntokens
    if log:
        print(f'Train: wpb={ntokens//niterations}, num_updates={niterations}, accuracy={accuracy:.1f}, loss={total_loss:.2f}')
    return accuracy, total_loss

In [29]:
def validate(model, criterion, idata, target, batch_size, device):
    model.eval()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    y_pred = []
    with torch.no_grad():
        for X, y in batch_generator(idata, target, batch_size, shuffle=False):
            # Get input and target sequences from batch
            X = torch.tensor(X, dtype=torch.long, device=device)
            output = model(X)
            if target is not None:
                y = torch.tensor(y, dtype=torch.long, device=device)
                loss = criterion(output, y)
                total_loss += loss.item()
                ncorrect += (torch.max(output, 1)[1] == y).sum().item()
                ntokens += y.numel()
                niterations += 1
            else:
                pred = torch.max(output, 1)[1].detach().to('cpu').numpy()
                y_pred.append(pred)

    if target is not None:
        total_loss = total_loss / ntokens
        accuracy = 100 * ncorrect / ntokens
        return accuracy, total_loss
    else:
        return np.concatenate(y_pred)

In [ ]:
def validate_SkipGram(model, criterion, idata, target, batch_size, device):
    model.eval()
    total_loss = 0
    ncorrect = 0
    ntokens = 0
    niterations = 0
    y_pred = []
    target = torch.from_numpy(target)
    target = target.float()
    with torch.no_grad():
        for X, y in batch_generator(idata, target, batch_size, shuffle=False):
            # Get input and target sequences from batch
            X = torch.tensor(X, dtype=torch.long, device=device)
            output = model(X)
             output = output.permute(0, 2, 1)
            if target is not None:
                y = torch.tensor(y, dtype=torch.long, device=device)
                loss = criterion(output, y)
                total_loss += loss.item()
                ncorrect += (output == y).sum().item()
                ntokens += y.numel()
                niterations += 1
            else:
                pred = torch.max(output, 1)[1].detach().to('cpu').numpy()
                y_pred.append(pred)

    if target is not None:
        total_loss = total_loss / ntokens
        accuracy = 100 * ncorrect / ntokens
        return accuracy, total_loss
    else:
        return np.concatenate(y_pred)

In [30]:
# Create working dir
pathlib.Path(WORKING_ROOT).mkdir(parents=True, exist_ok=True)

In [31]:
# Select device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    print("WARNING: Training without GPU can be very slow!")

In [32]:
vocab, data = load_preprocessed_dataset(params.preprocessed)

Number of samples (train): 82284341
Number of samples (valid): 164765
Number of samples (test): 165837
Using precomputed vocabulary and data files
Vocabulary size: 100002


In [33]:
"""        self.token2idx = {}
        self.idx2token = []
        self.pad_token = pad_token
        self.unk_token = unk_token
        self.eos_token = eos_token """
print(vocab.eos_token)

None


In [34]:
# 'El Periodico' validation dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_valid.csv')
tokens = valid_x_df.columns[1:]
valid_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')
valid_y_df = pd.read_csv(f'{COMPETITION_ROOT}/y_valid.csv')
valid_y = valid_y_df['token'].apply(vocab.get_index).to_numpy(dtype='int32')

In [35]:
# 'El Periodico' test dataset
valid_x_df = pd.read_csv(f'{COMPETITION_ROOT}/x_test.csv')
test_x = valid_x_df[tokens].apply(vocab.get_index).to_numpy(dtype='int32')

In [36]:
model = CBOW(len(vocab), params.embedding_dim).to(device)
print(model)
for name, param in model.named_parameters():
    print(f'{name:20} {param.numel()} {list(param.shape)}')
print(f'TOTAL                {sum(p.numel() for p in model.parameters())}')


CBOW(
  (emb): Embedding(100002, 400, padding_idx=0)
  (lin): Linear(in_features=400, out_features=100002, bias=False)
)
position_weight      2400 [6, 400]
emb.weight           40000800 [100002, 400]
lin.weight           40000800 [100002, 400]
TOTAL                80004000


In [ ]:
model = SkipGram(len(vocab), params.embedding_dim).to(device)
print(model)
for name, param in model.named_parameters():
    print(f'{name:20} {param.numel()} {list(param.shape)}')
print(f'TOTAL                {sum(p.numel() for p in model.parameters())}')

Note that $\textit{lin}$ layer transposes $\textit{lin.weight}$ before performing the multilplication.

The [nn.CrossEntropyLoss()](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html) criterion combines *nn.LogSoftmax()* and *nn.NLLLoss()* in one single class.

In [37]:
criterion = nn.CrossEntropyLoss(reduction='sum') #ya fa la softmax

In [ ]:
from torch.optim.lr_scheduler import OneCycleLR
optimizer = torch.optim.Adam(model.parameters())
scheduler = OneCycleLR(optimizer, max_lr=0.01, steps_per_epoch=len(vocab),
                       epochs=params.epochs, anneal_strategy='cos', cycle_momentum=False)
train_accuracy = []
wiki_accuracy = []
valid_accuracy = []
for epoch in range(params.epochs):
    acc, loss = train(model, criterion, optimizer,scheduler, data[0][0], data[0][1], params.batch_size, device, log=True)
    train_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | train accuracy={acc:.1f}%, train loss={loss:.2f}')
    acc, loss = validate(model, criterion, data[1][0], data[1][1], params.batch_size, device)
    wiki_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (wikipedia)')
    acc, loss = validate(model, criterion, valid_x, valid_y, params.batch_size, device)
    valid_accuracy.append(acc)
    print(f'| epoch {epoch:03d} | valid accuracy={acc:.1f}%, valid loss={loss:.2f} (El Periódico)')


# Save model
torch.save(model.state_dict(), params.modelname)

Train: wpb=1000, num_updates=200, accuracy=6.2, loss=9.84
Train: wpb=1000, num_updates=500, accuracy=11.4, loss=8.45
Train: wpb=1000, num_updates=1000, accuracy=15.0, loss=7.57
Train: wpb=1000, num_updates=2000, accuracy=18.5, loss=6.82
Train: wpb=1000, num_updates=3000, accuracy=20.6, loss=6.43
Train: wpb=1000, num_updates=4000, accuracy=22.2, loss=6.16
Train: wpb=1000, num_updates=5000, accuracy=23.4, loss=5.96
Train: wpb=1000, num_updates=6000, accuracy=24.5, loss=5.80
Train: wpb=1000, num_updates=7000, accuracy=25.4, loss=5.67
Train: wpb=1000, num_updates=8000, accuracy=26.2, loss=5.56
Train: wpb=1000, num_updates=9000, accuracy=27.0, loss=5.46
Train: wpb=1000, num_updates=10000, accuracy=27.6, loss=5.38
Train: wpb=1000, num_updates=11000, accuracy=28.2, loss=5.30
Train: wpb=1000, num_updates=12000, accuracy=28.7, loss=5.23
Train: wpb=1000, num_updates=13000, accuracy=29.2, loss=5.17
Train: wpb=1000, num_updates=14000, accuracy=29.7, loss=5.12
Train: wpb=1000, num_updates=15000, ac

In [ ]:
# Test prediction
y_pred = validate(model, None, test_x, None, params.batch_size, device)
y_token = [vocab.idx2token[index] for index in y_pred]

In [ ]:
submission = pd.DataFrame({'id':valid_x_df['id'], 'token': y_token}, columns=['id', 'token'])
print(submission.head())
submission.to_csv('submission.csv', index=False)